In [1]:
import cv2
from ultralytics import YOLOWorld
import supervision as sv

In [2]:
# Инициализируем модель
model = YOLOWorld("yolov8x-worldv2.pt")

# Задаем промпт для модели
classes = ["license_plate", ""]
model.set_classes(classes)

# Создаем датасет
dataset = sv.DetectionDataset(classes=classes, 
                              images=['437.jpg'], 
                              annotations={'437.jpg': None})

# Считываем изображение из папки
PATH_TO_IMAGES = "cars_images/images"
image = cv2.imread(f"{PATH_TO_IMAGES}/437.jpg")

# Запускаем детекцию с порогом уверенности
results = model.predict(image, conf=0.1, imgsz=(480, 640))

# Получение детекций для Supervision
detections = sv.Detections.from_ultralytics(results[0])

# фильтрация детекций по ширине bounding-box-а
w = detections.xyxy[:, 2] - detections.xyxy[:, 0]
h = detections.xyxy[:, 3] - detections.xyxy[:, 1]

# TODO подобрано эмпирически, надо разобраться
detections = detections[(w < 500)]

#  Non-Maximum Suppression (NMS) постобработка для удаления дублирующихся прямоугольников
filtered_detections = detections.with_nms(threshold=0.5)

# Инициализация экземпляров классов для боксов и подписей
annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# вывод доп информации (уверенность)
labels = [
    f"{class_name} {confidence:.2f}"
    for class_name, confidence
    in zip(filtered_detections['class_name'], filtered_detections.confidence)
]

# Непосредственно, рисуем боксы и подписи
annotated_image = annotator.annotate(scene = image, 
                                     detections = filtered_detections)
annotated_image = label_annotator.annotate(scene = annotated_image, 
                                           detections = filtered_detections, 
                                           labels=labels)

# Сохраняем размеченную картинку
cv2.imwrite("annotated_437.jpg", annotated_image)

# Вносим в датасет инфу по детекциям для конкретной картинки
dataset.annotations['437.jpg'] = filtered_detections

# Экспортируем в YOLO-формат (data_yaml генерируется невалидный для CVAT, нужно создавать самому)
dataset.as_yolo(
    images_directory_path="license_plate_dataset/images",
    annotations_directory_path="license_plate_dataset/labels",
)
